In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
import timedependence
import importlib
importlib.reload(timedependence)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
# load nue variables
NUEVARS = True
# USING BDT?
USEBDT = True

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

#urON = uproot.open(ls.ONBEAM_NUMU_PATH)["nuselection"][tree] # numus
urON = uproot.open(ls.ONBEAM_FAR_PATH)["nuselection"][tree]
urOF = uproot.open(ls.OFFBEAM_PATH)["nuselection"][tree]

variables_nuepresl = [
    "_opfilter_pe_beam", "_opfilter_pe_veto", # did the event pass the common optical filter (for MC only)
    "n_showers_contained","selected","trk_llr_pid_score_v","trk_id","shr_id",
    "run", "sub", "evt", 
    "slnunhits", "slnhits", 
   "tksh_distance", 
    "shr_energy_tot_cali", "nslice", 
]

variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category",
    "shr_tkfit_dedx_U","shr_tkfit_dedx_V","shr_tkfit_dedx_Y",
    "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
    "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy2",
    "shr_px","shr_py","shr_pz",
    #"run","sub","evt",
    "CosmicIP","CosmicDirAll3D","CosmicIPAll3D",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist2", # "trkshrhitdist0","trkshrhitdist1", distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    #"pi0_energy2_Y", # pi0 tagger variables
    "_opfilter_pe_beam", "_opfilter_pe_veto", # did the event pass the common optical filter (for MC only)
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2",
    "shr_tkfit_nhits_Y","shr_tkfit_nhits_U","shr_tkfit_nhits_V",
    "shr_tkfit_2cm_nhits_Y","shr_tkfit_2cm_nhits_U","shr_tkfit_2cm_nhits_V",
    "shr_tkfit_gap10_nhits_Y","shr_tkfit_gap10_nhits_U","shr_tkfit_gap10_nhits_V",
    # pi0 variables
    "pi0_radlen1","pi0_radlen2","pi0_dot1","pi0_dot2","pi0_energy1_Y","pi0_energy2_Y",
    "pi0_dedx1_fit_Y","pi0_dedx2_fit_Y","pi0_shrscore1","pi0_shrscore2","pi0_gammadot",
    "pi0_dedx1_fit_V","pi0_dedx2_fit_V","pi0_dedx1_fit_U","pi0_dedx2_fit_U",
    "pi0_mass_Y","pi0_mass_V","pi0_mass_U",
    "pi0_dir2_x","pi0_dir2_y","pi0_dir2_z","pi0_dir1_x","pi0_dir1_y","pi0_dir1_z", 
    "trk_start_x_v","trk_start_y_v","trk_start_z_v",
    "shr_start_x_v","shr_start_y_v","shr_start_z_v"
]

variables_pi0 = [
    "run",'sub','evt','nslice',
    # pi0 variables
    "pi0_radlen1","pi0_radlen2","pi0_dot1","pi0_dot2","pi0_energy1_Y","pi0_energy2_Y",
    "pi0_dedx1_fit_Y","pi0_dedx2_fit_Y","pi0_shrscore1","pi0_shrscore2","pi0_gammadot",
    "pi0_dedx1_fit_V","pi0_dedx2_fit_V","pi0_dedx1_fit_U","pi0_dedx2_fit_U",
    "pi0_mass_Y","pi0_mass_V","pi0_mass_U",
    "pi0_dir2_x","pi0_dir2_y","pi0_dir2_z","pi0_dir1_x","pi0_dir1_y","pi0_dir1_z",
    'reco_nu_vtx_sce_x','reco_nu_vtx_sce_y','reco_nu_vtx_sce_z',
    'NeutrinoEnergy2','NeutrinoEnergy1','NeutrinoEnergy0'
]

#variables_pi0 = variables_nuepresl + variables_pi0

#make the list unique
variables = list(set(variables))
#print(variables)

variables.remove("_closestNuCosmicDist")
variables.remove("crtveto")
variables.remove("crthitpe")

dfON = urON.pandas.df(variables, flatten=False)
dfOF = urOF.pandas.df(variables, flatten=False)

if (NUEVARS):

    uproot_v = [urON,urOF]
    df_v = [dfON,dfOF]
    for i,df in enumerate(df_v):
        up = uproot_v[i]
        trk_llr_pid_v = up.array('trk_llr_pid_score_v')
        trk_start_x_v = up.array('trk_start_x_v')
        trk_start_y_v = up.array('trk_start_y_v')
        trk_start_z_v = up.array('trk_start_z_v')
        shr_start_x_v = up.array('shr_start_x_v')
        shr_start_y_v = up.array('shr_start_y_v')
        shr_start_z_v = up.array('shr_start_z_v')
        #trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
        trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
        shr_id = up.array('shr_id')-1 # I think we need this -1 to get the right result
        trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
        #trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
        df['trkpid'] = trk_llr_pid_v_sel
        shr_xs = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(shr_start_x_v,shr_id)])
        shr_ys = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(shr_start_y_v,shr_id)])
        shr_zs = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(shr_start_z_v,shr_id)])
        shr_xt = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_start_x_v,shr_id)])
        shr_yt = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_start_y_v,shr_id)])
        shr_zt = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_start_z_v,shr_id)])
        trk_xt = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_start_x_v,trk_id)])
        trk_yt = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_start_y_v,trk_id)])
        trk_zt = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_start_z_v,trk_id)])
        df['shrxt'] = shr_xt
        df['shryt'] = shr_yt
        df['shrzt'] = shr_zt
        df['shrxs'] = shr_xs
        df['shrys'] = shr_ys
        df['shrzs'] = shr_zs
        df['trkxt'] = trk_xt
        df['trkyt'] = trk_yt
        df['trkzt'] = trk_zt
        #df['trackcaloenergy'] = trk_calo_energy_y_sel

In [ ]:
# how to get the LLR-PID value for the "track candidate" (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!

INTERCEPT = 0.0
SLOPE = 0.83

df_v = [dfON,dfOF]

if (NUEVARS):

    for i,df in enumerate(df_v):
        
        #'''
        df['tkshdxt'] = df['shrxt'] - df['trkxt']
        df['tkshdyt'] = df['shryt'] - df['trkyt']
        df['tkshdzt'] = df['shrzt'] - df['trkzt']
        
        df['tkshdxs'] = df['shrxs'] - df['trkxt']
        df['tkshdys'] = df['shrys'] - df['trkyt']
        df['tkshdzs'] = df['shrzs'] - df['trkzt']
        
        df['shrptot'] = np.sqrt( (df['shr_px'])**2 + (df['shr_py'])**2 + (df['shr_pz'])**2  )
        df['shrpzfrac'] = df['shr_pz'] / df['shrptot']
        df['shrpyfrac'] = df['shr_py'] / df['shrptot']
        df['shrpxfrac'] = df['shr_px'] / df['shrptot']
        
        df['tksh3dt'] = np.sqrt( (df['tkshdxt'])**2 + (df['tkshdyt'])**2 + (df['tkshdzt'])**2 )
        df['tksh3ds'] = np.sqrt( (df['tkshdxs'])**2 + (df['tkshdys'])**2 + (df['tkshdzs'])**2 )
        #'''
        
        df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
        df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
        # and the 2d angle difference
        df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])
        df['shr_tkfit_nhits_tot'] = (df['shr_tkfit_nhits_Y']+df['shr_tkfit_nhits_U']+df['shr_tkfit_nhits_V'])
        df['shr_tkfit_dedx_avg'] = (df['shr_tkfit_nhits_Y']*df['shr_tkfit_dedx_Y'] + df['shr_tkfit_nhits_U']*df['shr_tkfit_dedx_U'] + df['shr_tkfit_nhits_V']*df['shr_tkfit_dedx_V'])/df['shr_tkfit_nhits_tot']
        df['shr_tkfit_2cm_nhits_tot'] = (df['shr_tkfit_2cm_nhits_Y']+df['shr_tkfit_2cm_nhits_U']+df['shr_tkfit_2cm_nhits_V'])
        df['shr_tkfit_2cm_dedx_avg'] = (df['shr_tkfit_2cm_nhits_Y']*df['shr_tkfit_2cm_dedx_Y'] + df['shr_tkfit_2cm_nhits_U']*df['shr_tkfit_2cm_dedx_U'] + df['shr_tkfit_2cm_nhits_V']*df['shr_tkfit_2cm_dedx_V'])/df['shr_tkfit_2cm_nhits_tot']
        df['shr_tkfit_gap10_nhits_tot'] = (df['shr_tkfit_gap10_nhits_Y']+df['shr_tkfit_gap10_nhits_U']+df['shr_tkfit_gap10_nhits_V'])
        df['shr_tkfit_gap10_dedx_avg'] = (df['shr_tkfit_gap10_nhits_Y']*df['shr_tkfit_gap10_dedx_Y'] + df['shr_tkfit_gap10_nhits_U']*df['shr_tkfit_gap10_dedx_U'] + df['shr_tkfit_gap10_nhits_V']*df['shr_tkfit_gap10_dedx_V'])/df['shr_tkfit_gap10_nhits_tot']
        df.loc[:,'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_Y']
        df.loc[(df['shr_tkfit_nhits_U']>df['shr_tkfit_nhits_Y']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_U']
        df.loc[(df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_Y']) & (df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_U']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_V']
        df.loc[df['secondshower_Y_dot'].isna(),'secondshower_Y_dot'] = 0.0
        df["reco_e"] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + df["trk_energy_tot"]
        df["reco_e_qe"] = 0.938*((df["shr_energy"]+INTERCEPT)/SLOPE)/(0.938 - ((df["shr_energy"]+INTERCEPT)/SLOPE)*(1-np.cos(df["shr_theta"])))
        df["reco_e_rqe"] = df["reco_e_qe"]/df["reco_e"]
        df["bnbdata"] = np.zeros_like(df["shr_energy"])
        df["extdata"] = np.zeros_like(df["shr_energy"])
        df.loc[(df['category']!=1)&(df['category']!=10)&(df['category']!=11)&(df['category']!=111)&(df['slnunhits']/df['slnhits']<0.2), 'category'] = 4

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
import matplotlib.gridspec as gridspec

In [ ]:
fig = plt.figure(figsize=(6,6))
BINS2D = (np.linspace(0,10,100),np.linspace(0,10,1000))
BINS1D = np.linspace(10,300,20)
dfONsub = dfON.query('run < 12000')
plt.hist(dfONsub['pi0_mass_Y'].values,bins=BINS1D,histtype='step',lw=2,normed=True,label='run < 12k')
dfONsub = dfON.query('run > 12000')
plt.hist(dfONsub['pi0_mass_Y'].values,bins=BINS1D,histtype='step',lw=2,normed=True,label='run > 12k')
#plt.yscale('log')
plt.legend(loc=1)
plt.show()

## fig = plt.figure(figsize=(6,6))
BINS2D = (np.linspace(0,10,100),np.linspace(0,10,1000))
BINS1D = np.linspace(-10,10,50)
dfONsub = dfON.query('run < 12000')
plt.hist(dfONsub['tkshdys'].values,bins=BINS1D,histtype='step',lw=2,normed=True,label='run < 12k')
dfONsub = dfON.query('run > 12000')
plt.hist(dfONsub['tkshdys'].values,bins=BINS1D,histtype='step',lw=2,normed=True,label='run > 12k')
#plt.yscale('log')
plt.legend()
plt.show()


In [ ]:
fig = plt.figure(figsize=(6,6))
BINS2D = (np.linspace(0,10,100),np.linspace(0,10,1000))
BINS1D = np.linspace(-0.3,0.3,100)
dfONsub = dfON.query('run > 12000 and tksh_distance < 10.')
res1d = (dfONsub['tksh3dt'].values-dfONsub['tksh3ds'].values)#/dfONsub['tksh3d'].values
#res1d = (dfONsub['tksh3dt'].values-dfONsub['tksh_distance'].values)#/dfONsub['tksh3d'].values
#plt.hist2d(dfONsub['tksh3d'].values,dfONsub['tksh_distance'].values,bins=BINS2D)
plt.hist(res1d,bins=BINS1D,histtype='step',lw=2,normed=True,label='run > 12k')
dfONsub = dfON.query('run < 12000  and tksh_distance < 10.')
res1d = (dfONsub['tksh3dt'].values-dfONsub['tksh3ds'].values)#/dfONsub['tksh3d'].values
#res1d = (dfONsub['tksh3dt'].values-dfONsub['tksh_distance'].values)#/dfONsub['tksh3d'].values
#plt.hist2d(dfONsub['tksh3d'].values,dfONsub['tksh_distance'].values,bins=BINS2D)
plt.hist(res1d,bins=BINS1D,histtype='step',lw=2,normed=True,label='run < 12k')
#plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
#df_v = [dfON,dfOF]
#for i,df in enumerate(df_v):
#    df["bnbdata"] = np.zeros_like(df["run"])
#    df["extdata"] = np.zeros_like(df["run"])
dfON['bnbdata'] = np.ones_like(dfON["nslice"])
dfOF['extdata'] = np.ones_like(dfOF["nslice"])

In [ ]:
# variables to be trained on
TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max",
            "trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "CosmicIPAll3D","CosmicDirAll3D"]

In [ ]:
LABELS =  ['pi0','nonpi0']
#LABELS =  ["bkg"]

if (USEBDT == True):
    for label, bkg_query in zip(LABELS, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_0304_extnumi.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)

            dfOF[label+"_score"] = booster.predict(
                xgb.DMatrix(dfOF[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            dfON[label+"_score"] = booster.predict(
                xgb.DMatrix(dfON[TRAINVAR]),
                ntree_limit=booster.best_iteration)

In [ ]:
import timedependence
import importlib
importlib.reload(timedependence)

In [ ]:
# nue files
timeplotter = timedependence.TimePlotter(ls.ONBEAM_FAR_POT,ls.OFFBEAM_POT,dfON,dfOF)
# pi0 / numu files
#timeplotter = timedependence.TimePlotter(ls.ONBEAM_NUMU_POT,ls.OFFBEAM_POT,dfON,dfOF)

In [ ]:
import unblinding_far_sideband
import importlib
importlib.reload(unblinding_far_sideband)

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def line(x,s,b):
    return x*s + b

In [ ]:
#print (PRESQ_twoplus_showers)
#QUERY = unblinding_far_sideband.PRESQ_twoplus_showers
#QUERY =  unblinding_far_sideband.NPPRESEQ_one_shower 
QUERY = unblinding_far_sideband.NPVLCUTQ
#QUERY = unblinding_far_sideband.stages_queries[3]
#QUERY = unblinding_far_sideband.PI0SEL
#QUERY = ' nslice == 1 and NeutrinoEnergy2 > 1000.'# and NeutrinoEnergy2 < 1000.'

QUERY += ' and n_showers_contained == 1'

TITLE = 'NPVLCUTQ and n_showers_contained == 1'

#QUERY += ' and n_tracks_contained == 1 and n_showers_contained == 1'

#QUERY += ' and  n_tracks_contained == 1 and reco_e > 0.9'

# two-shower selection
#QUERY += ' and n_showers_contained == 2 and trkpid > 0.2'


#QUERYGEN = '(1.05 < reco_e < 2.05) and nslice == 1 and selected == 1  and shr_energy_tot_cali > 0.07'
#QUERYGEN += ' and n_tracks_contained != 1 and n_showers_contained == 1'
#QUERY = QUERYGEN

#QUERY += ' and CosmicIPAll3D > 10.'
#QUERY += ' and trkpid < 0.25'
#QUERY += ' and hits_ratio > 0.5'
#QUERY += ' and trkfit < 0.90'
#QUERY += ' and tksh_distance < 10.0'
#QUERY += ' and tksh_angle > -0.9'


#QUERY += ' and n_tracks_contained == 1 and n_showers_contained == 1'

#QUERY = '(1.05 < reco_e < 2.05) and nslice == 1 and selected == 1  and shr_energy_tot_cali > 0.07 and ( (_opfilter_pe_beam > 0 and _opfilter_pe_veto < 20) or bnbdata == 1 or extdata == 1) and trkfit < 0.90'

#QUERY = 'nslice == 1 and selected == 1 and shr_energy_tot_cali > 0.07 and ( (_opfilter_pe_beam > 0 and _opfilter_pe_veto < 20) or bnbdata == 1 or extdata == 1) and n_tracks_contained > 0 and CosmicIPAll3D > 10. and hits_ratio > 0.5 and tksh_distance < 10.0 and tksh_angle > -0.9'

#QUERY += ' and reco_nu_vtx_sce_x > 150.'

#QUERY += ' and (shr_energy_tot_cali > 0.2)'
#TITLE += '\nand shr_energy_tot_cali > 0.2'
print (QUERY)

#TITLE = '1eNp pre-selection tracks != 1 and 1-shower'
#TITLE += '\nadding cut : NONE'

ONBEAM = True

In [ ]:
RUN_BOUNDS = np.array([4952,8317,13696,18960])
#RUN_BOUNDS = np.array([300])

#run_v, ONpass_v, ONpass_e = timeplotter.QueryByRun(RUN_BOUNDS,QUERY,ONBEAMONLY=True,POTmin=10)
run_v, pass_v, pass_e, ONpass_v, ONpass_e, OFpass_v, OFpass_e, selON_v = timeplotter.QueryByRun(RUN_BOUNDS,QUERY,ONBEAMONLY=False,POTmin=0)#,RMIN=13700,RMAX=18000)

In [ ]:
fig = plt.figure(figsize=(12,10))

gs = fig.add_gridspec(3,1)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax3 = fig.add_subplot(gs[2])
#ax2 = ax1.twinx()

sONrun_v = []
sONpass_v = []
sONpass_e = []
srun_v = []
spass_v = []
spass_e = []

sOFrun_v = []
sOFpass_v = []
sOFpass_e = []


for i,n in enumerate(selON_v):
    if ( (n > 0) and (ONpass_v[i] != np.isnan) and (ONpass_v[i] > 0) and ( np.isinf(ONpass_v[i]) == False) ):
        sONrun_v.append(run_v[i])
        sONpass_v.append(ONpass_v[i])
        sONpass_e.append(ONpass_e[i])
        if (OFpass_v[i] > 0):
            srun_v.append(run_v[i])
            spass_v.append(pass_v[i])
            spass_e.append(pass_e[i])
        
for i,n in enumerate(OFpass_v):
    if (n > 0):
        sOFrun_v.append(run_v[i])
        sOFpass_v.append(OFpass_v[i])
        sOFpass_e.append(OFpass_e[i])

#ONrun_v = np.select([selON_v > 0],[run_v])
#ONpass_e = np.select([selON_v > 0],[ONpass_e])
#ONpass_v = np.select([selON_v > 0],[ONpass_v])
                                    
#ONrun_v = np.select([selON_v > 0],[run_v])
#pass_e = np.select([selON_v > 0],[pass_e])
#pass_v = np.select([selON_v > 0],[pass_v])

ax3.errorbar(srun_v,spass_v,yerr=spass_e,fmt='^',color='k',label='on-beam')
ax1.errorbar(sONrun_v,sONpass_v,yerr=sONpass_e,fmt='.',color='r',label='on-beam')
ax2.errorbar(sOFrun_v,sOFpass_v,yerr=sOFpass_e,fmt='*',color='b',label='off-beam')

ax1.set_xticklabels([])
ax2.set_xticklabels([])

ax1.set_xlim([np.min(run_v)-100,np.max(run_v)+100])
ax2.set_xlim([np.min(run_v)-100,np.max(run_v)+100])
ax3.set_xlim([np.min(run_v)-100,np.max(run_v)+100])


#plt.errorbar(run_v,pass_v,yerr=pass_e,fmt='o',color='b',label='(on-off) beam')
#plt.legend()

meanON = np.mean(sONpass_v)
stdON = np.std(sONpass_v)

meanOF = np.mean(sOFpass_v)
stdOF = np.std(sOFpass_v)

meanDIFF = np.mean(spass_v)
stdDIFF = np.std(spass_v)

print (meanON)
print (meanOF)
print (meanDIFF)


#print (pass_e)
'''
popt,popv = curve_fit(line,run_v,pass_v,sigma=pass_e,absolute_sigma=True,p0=[0.,np.mean(pass_v)])
pope = np.sqrt(np.diag(popv))
slopeval = (popt[0]/popt[1])*1e3 # fractional change / 1k runs
slopeerr = (pope[0]/popt[1])*1e3
print ('fits : ',popt)
print ('errors : ',pope)
percentdiff  = 100. * ( (line(np.max(run_v),*popt)) - (line(np.min(run_v),*popt)) ) / (line(np.min(run_v),*popt))
percenterrUP = 100. * ( (line(np.max(run_v),popt[0]+pope[0],popt[1])) - (line(np.min(run_v),popt[0]+pope[0],popt[1])) ) / (line(np.min(run_v),popt[0]+pope[0],popt[1]))
percenterrDN = 100. * ( (line(np.max(run_v),popt[0]-pope[0],popt[1])) - (line(np.min(run_v),popt[0]-pope[0],popt[1])) ) / (line(np.min(run_v),popt[0]-pope[0],popt[1]))
#plt.plot(run_v,line(run_v,*popt),'b--',lw=2,label='change: %.0f [%.0f, %.0f] %%'%(percentdiff,percenterrDN,percenterrUP))
#plt.ylim([0,14])
'''
ax1.grid()
ax2.grid()
ax3.grid()
plt.tight_layout()
ax1.set_title(TITLE,loc='left',fontsize=18)
#plt.legend(loc='best',fontsize=16)
plt.xlabel('run number')
#plt.ylim([.0,1000])

NS = 3.

#ax1.set_ylim([meanON-NS*stdON,meanON+NS*stdON])
#ax2.set_ylim([meanOF-NS*stdOF,meanOF+NS*stdOF])
#ax3.set_ylim([meanDIFF-NS*stdDIFF,meanDIFF+NS*stdDIFF])

ax1.tick_params(axis='y', labelcolor='tab:red')
for tick in ax1.yaxis.get_major_ticks():
    tick.label.set_fontsize(14) 
ax1.set_ylabel('on-beam / 1e18 POT',color='tab:red',fontsize=14)

ax2.tick_params(axis='y', labelcolor='b')
for tick in ax2.yaxis.get_major_ticks():
    tick.label.set_fontsize(14) 
ax2.set_ylabel('off-beam / 1e5 EXT',color='b',fontsize=14)

ax3.tick_params(axis='y', labelcolor='k')
for tick in ax3.yaxis.get_major_ticks():
    tick.label.set_fontsize(14) 
ax3.set_ylabel('(on-off) / 1e18 POT',color='k',fontsize=14)


plt.subplots_adjust(wspace=0, hspace=0)


#plt.ylim([-500,2000])
#if (ONBEAM == True):
#    plt.ylabel('events / 1e18 POT',fontsize=16)
#else:
#    plt.ylabel('(on-off) beam / 1e18 POT',fontsize=16)
plt.show()